In [1]:
import boto3 

region = 'us-east-1'
ec2 = boto3.client('ec2', region_name=region)

In [2]:
#get group ids from list of all security groups

def get_all_security_group_ids() -> list:
  return [sg.get('GroupId')for sg in ec2.describe_security_groups().get('SecurityGroups')]

In [3]:
get_all_security_group_ids()

['sg-0a417993a81248f14',
 'sg-0e77db9c3f1ae0b5d',
 'sg-0dd2cf5e7c4d71ad8',
 'sg-023ff0a431cf2d9df',
 'sg-03bcfec048792e00b']

___

In [4]:
# get security group ids for all security groups attached to an instance
# keeping in mind an instance could have more that one sg attached

def get_attached_security_group_ids() -> list:
  all_instances = ec2.describe_instances().get('Reservations')
  security_groups = [i.get('Instances')[0]['SecurityGroups'] for i in all_instances]
  return [ i[j].get('GroupId') for i in security_groups for j in range(0, len(i))]

In [5]:
get_attached_security_group_ids()

['sg-03bcfec048792e00b', 'sg-0a417993a81248f14', 'sg-023ff0a431cf2d9df']

___

In [6]:
#filter all sg id list for ids not in attached sg id list

def get_unattached_sg_ids() -> list:
  return list(filter(lambda id: id not in get_attached_security_group_ids(), get_all_security_group_ids()))

In [7]:
get_unattached_sg_ids()

['sg-0e77db9c3f1ae0b5d', 'sg-0dd2cf5e7c4d71ad8']

_____

`if you run this command below it will delete your unsattached SGs`

In [12]:
# delete unattached sgs

def delete_unattached_sg(list_of_unattached_sgs):
  for sg in list_of_unattached_sgs:
    try:
      ec2.delete_security_group(GroupId=sg)
    except Exception as e:
      if '"default" cannot be deleted by a user' in str(e):
        print(f"Unable to delete {sg}, check that it isn't the default sg or referenced by another sg")

In [13]:
def init():
    delete_unattached_sg(get_unattached_sg_ids())



if __name__ == "__main__":
    init()

Unable to delete sg-0dd2cf5e7c4d71ad8, check that it isn't the default sg or referenced by another sg
